In [ ]:
import pandas as pd

df = pd.read_csv('/content/IOT-temp 2.csv')

df=df.drop(columns = ['id', 'room_id/id', 'noted_date'])
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
#df = imputer.fit_transform(df)

from sklearn.preprocessing import LabelEncoder
le1= LabelEncoder()


df['out/in']= le1.fit_transform(df['out/in'])

'''
df=df.dropna()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le1 = LabelEncoder()
le2 = LabelEncoder()
'''


#df['icon']= le.fit_transform(df['icon'])

y = df['out/in'].values
df=df.drop(columns = ['out/in'])
X = df.values


In [ ]:

y=y.reshape(-1, 1)


import numpy as np
X=np.concatenate((X[1:], y[0:-1]), axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X= sc.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(X, y[1:], test_size=0.2, shuffle=False, random_state=42)


In [ ]:
x_train1 = x_train1.reshape((x_train1.shape[0], x_train1.shape[1], 1))
x_test1 = x_test1.reshape((x_test1.shape[0], x_test1.shape[1], 1))

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(2, activation="softmax")(x)
    #outputs = layers.Dense(1)(x)


    return keras.Model(inputs, outputs)

In [ ]:


input_shape = x_train1.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)


model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["sparse_categorical_accuracy"],
)
'''
model.compile(
    loss="mean_absolute_error",
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["mean_absolute_error"],
)
'''
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2, 1)]       0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 2, 1)        2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 2, 1)        7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

In [ ]:


callbacks = [keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)]

history=model.fit(
    x_train1,
    y_train1,
    validation_data=(x_test1, y_test1),
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
)


Epoch 1/100
611/611 [==============================] - 14s 23ms/step - loss: 0.2228 - sparse_categorical_accuracy: 0.8880 - val_loss: 0.6571 - val_sparse_categorical_accuracy: 0.5184
Epoch 2/100
611/611 [==============================] - 8s 13ms/step - loss: 0.2222 - sparse_categorical_accuracy: 0.8881 - val_loss: 0.6489 - val_sparse_categorical_accuracy: 0.6003
Epoch 3/100
611/611 [==============================] - 8s 13ms/step - loss: 0.2225 - sparse_categorical_accuracy: 0.8880 - val_loss: 0.6498 - val_sparse_categorical_accuracy: 0.5099
Epoch 4/100
611/611 [==============================] - 8s 13ms/step - loss: 0.2222 - sparse_categorical_accuracy: 0.8886 - val_loss: 0.6497 - val_sparse_categorical_accuracy: 0.6003
Epoch 5/100
611/611 [==============================] - 8s 13ms/step - loss: 0.2225 - sparse_categorical_accuracy: 0.8891 - val_loss: 0.6518 - val_sparse_categorical_accuracy: 0.6003
Epoch 6/100
611/611 [==============================] - 8s 13ms/step - loss: 0.2231 - spar

In [ ]:
#testing prediction
ypred1=model.predict(x_test1)
ypred=[]
#testing prediction
for i in ypred1:
    if i[0]>i[1]:
        ypred.append(0)
    else:
        ypred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print('accuracy', accuracy_score(ypred, y_test1))



print('f1_score', f1_score(ypred, y_test1))



print('jaccard_score', jaccard_score(ypred, y_test1))
print('roc_auc_score', roc_auc_score(ypred, y_test1))
print('precision_score', precision_score(ypred, y_test1))

print('recall_score', recall_score(ypred, y_test1))


accuracy 0.6492495261513243
f1_score 0.6884753628463532
jaccard_score 0.5249427600083258
roc_auc_score 0.6464247508949346
precision_score 0.6322914925622597
recall_score 0.7556176969939079


In [ ]:
#ttraining prediction
xpred1=model.predict(x_train1)
xpred=[]

for i in xpred1:
    if i[0]>i[1]:
        xpred.append(0)
    else:
        xpred.append(1)
from sklearn.metrics import accuracy_score, f1_score,  jaccard_score, roc_auc_score, precision_score, recall_score
print('accuracy', accuracy_score(xpred, y_train1))



print('f1_score', f1_score(xpred, y_train1))



print('jaccard_score', jaccard_score(xpred, y_train1))
print('roc_auc_score', roc_auc_score(xpred, y_train1))
print('precision_score', precision_score(xpred, y_train1))

print('recall_score', recall_score(xpred, y_train1))


accuracy 0.8881717125147277
f1_score 0.9345309500959693
jaccard_score 0.8771075520027022
roc_auc_score 0.8090993744458292
precision_score 0.9544681828623938
recall_score 0.9154095856406339
